In [148]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.sql import text
from datetime import date, datetime

DB_NAME = "publictransportation"
DB_USER = "postgres"
DB_PASS = "hellosql"
DB_HOST = "localhost"
DB_PORT = "5432"

In [149]:
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}")
conn = engine.connect()

sql_query = """
CREATE TABLE if not exists monthly_totals
(id serial,
year int,
month varchar,
city varchar,
transportation_type varchar,
ridership_count int);
"""

conn.execute(text(sql_query))

In [150]:
chicago_df = pd.read_csv('CTA_-_Ridership_-_Daily_Boarding_Totals_20240307.csv')
newyork_df = pd.read_csv('MTA_Monthly_Ridership___Traffic_Data__Beginning_January_2008.csv')

In [151]:
chicago_df['service_date'] = pd.to_datetime(chicago_df['service_date'])

In [152]:
newyork_df['Month'] = pd.to_datetime(newyork_df['Month'])

In [153]:
chicago_df = chicago_df.drop(columns = ['day_type', 'total_rides'])

In [154]:
new_chicago_df = pd.melt(chicago_df, id_vars = ['service_date'], value_vars = ['bus', 'rail_boardings'], var_name = 'transportation_type', value_name = 'ridership_count')

In [155]:
new_chicago_df['transportation_type'].mask(new_chicago_df['transportation_type'] == 'rail_boardings', 'metro', inplace = True)

In [156]:
new_chicago_df['city'] = "Chicago"

In [157]:
def find_month(date):
    if date.month == 1:
        return 'January'
    elif date.month == 2:
        return 'February'
    elif date.month == 3:
        return 'March'
    elif date.month == 4:
        return 'April'
    elif date.month == 5:
        return 'May'
    elif date.month == 6:
        return 'June'
    elif date.month == 7:
        return 'July'
    elif date.month == 8:
        return 'August'
    elif date.month == 9:
        return 'September'
    elif date.month == 10:
        return 'October'
    elif date.month == 11:
        return 'November'
    elif date.month == 12:
        return 'December'
    else:
        return 'Invalid month'
new_chicago_df['month'] = new_chicago_df['service_date'].apply(find_month)

In [158]:
new_chicago_df['year'] = new_chicago_df['service_date'].apply(lambda date: date.year)

In [159]:
final_chicago_df = new_chicago_df.drop(columns = ['service_date'])
final_chicago_df = final_chicago_df.groupby(['year', 'month', 'transportation_type', 'city'])['ridership_count'].sum().reset_index()

display(final_chicago_df)

,year,month,transportation_type,city,ridership_count
0,2001,April,bus,Chicago,25053791
1,2001,April,metro,Chicago,14713336
2,2001,August,bus,Chicago,24647188
3,2001,August,metro,Chicago,16014736
4,2001,December,bus,Chicago,23428582
...,...,...,...,...,...
543,2023,May,metro,Chicago,10157092
544,2023,October,bus,Chicago,15242489
545,2023,October,metro,Chicago,11509844
546,2023,September,bus,Chicago,14796427


## New York

In [160]:
newyork_df['city'] = "New York"

In [161]:
newyork_df['month'] = newyork_df['Month'].apply(find_month)

In [162]:
newyork_df['year'] = newyork_df['Month'].apply(lambda date: date.year)

In [163]:
newyork_df = newyork_df.rename(columns = {"Ridership": "ridership_count"})

In [164]:
newyork_df = newyork_df[newyork_df['Agency'] != "B&T"]

In [165]:
def transport_type(agency):
    if agency == "MNR" or agency == "Subway" or agency == "LIRR" or agency == "SIR":
        return "metro"
    else:
        return "bus"
    

In [166]:
newyork_df['transportation_type'] = newyork_df['Agency'].apply(transport_type)

In [167]:

final_newyork_df = newyork_df.groupby(['transportation_type','year', 'month', 'city'])['ridership_count'].sum().reset_index()
display(final_newyork_df)

,transportation_type,year,month,city,ridership_count
0,bus,2018,April,New York,48544481
1,bus,2018,August,New York,46810522
2,bus,2018,December,New York,44033258
3,bus,2018,February,New York,44719811
4,bus,2018,January,New York,45398837
...,...,...,...,...,...
261,metro,2023,May,New York,110376738
262,metro,2023,November,New York,104558133
263,metro,2023,October,New York,110197921
264,metro,2023,September,New York,101009689


In [168]:
combined_df = pd.concat([final_chicago_df, final_newyork_df])
display(combined_df)

,year,month,transportation_type,city,ridership_count
0,2001,April,bus,Chicago,25053791
1,2001,April,metro,Chicago,14713336
2,2001,August,bus,Chicago,24647188
3,2001,August,metro,Chicago,16014736
4,2001,December,bus,Chicago,23428582
...,...,...,...,...,...
261,2023,May,metro,New York,110376738
262,2023,November,metro,New York,104558133
263,2023,October,metro,New York,110197921
264,2023,September,metro,New York,101009689


In [170]:
for index, row in combined_df.iterrows():
    values = tuple(row)
    query = f"insert into monthly_totals (year, month, transportation_type, city, ridership_count) values {values}"
    conn.execute(query)